<a href="https://colab.research.google.com/github/bijoy3d/BigDataProjectCS6350_NWMB/blob/master/Homework_ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
---
NLP
Homework : Authorship Attribution of the Federalist Papers
---
Bijoy Prakash
---
BXP200013

---
---

In [223]:
from google.colab import files
uploaded = files.upload()

Saving federalist.csv to federalist (2).csv


Import all the necessary packages

In [224]:
import io
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

---
**1. Read in the csv file using pandas**

---

In [225]:
df = pd.read_csv(io.BytesIO(uploaded['federalist.csv']))#, index_col='ID')

In [226]:
df.head()

,author,text
0,HAMILTON,FEDERALIST. No. 1 General Introduction For the...
1,JAY,FEDERALIST No. 2 Concerning Dangers from Forei...
2,JAY,FEDERALIST No. 3 The Same Subject Continued (C...
3,JAY,FEDERALIST No. 4 The Same Subject Continued (C...
4,JAY,FEDERALIST No. 5 The Same Subject Continued (C...


In [227]:
df.dtypes

author    object
text      object
dtype: object

---
**1. Convert the author column to categorical data.**

---

In [228]:
df['author'] = df['author'].astype('category')
df.dtypes

author    category
text        object
dtype: object

---
**1.Display the first few rows.**

---

In [229]:
df.head()

,author,text
0,HAMILTON,FEDERALIST. No. 1 General Introduction For the...
1,JAY,FEDERALIST No. 2 Concerning Dangers from Forei...
2,JAY,FEDERALIST No. 3 The Same Subject Continued (C...
3,JAY,FEDERALIST No. 4 The Same Subject Continued (C...
4,JAY,FEDERALIST No. 5 The Same Subject Continued (C...


---
**1. Display the counts by author.**

---

In [230]:
df[['author']].value_counts().reset_index(name='count')

,author,count
0,HAMILTON,49
1,MADISON,15
2,HAMILTON OR MADISON,11
3,JAY,5
4,HAMILTON AND MADISON,3


---
**2. Divide into train and test, with 80% in train. Use random state 1234. Display the shape of train and test.**

---

In [231]:
X=df.text
y=df.author
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=1234)
print(f'X_train shape: {X_train.shape}\nX_test shape: {X_test.shape}')

X_train shape: (66,)
X_test shape: (17,)


---
**3. Process the text by removing stop words and performing tf-idf vectorization, fit to the training data only, and applied to train and test. Output the training set shape and the test set shape.**

---

In [232]:
vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)
X_vtrain = vectorizer.fit_transform(X_train)
X_vtest = vectorizer.transform(X_test)
print(f'X_train shape: {X_vtrain.shape}\nX_test shape: {X_vtest.shape}')

X_train shape: (66, 7727)
X_test shape: (17, 7727)


---
**4. Try a Bernoulli Naïve Bayes model. What is your accuracy on the test set?**

---

In [233]:
naive_bayes = BernoulliNB()
naive_bayes.fit(X_vtrain, y_train)
pred = naive_bayes.predict(X_vtest)
print(pred)
print('\nBernoulli Naïve Bayes model Accuracy is: ', accuracy_score(y_test, pred))

['HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON']

Bernoulli Naïve Bayes model Accuracy is:  0.5882352941176471


---
**5. The results from step 4 will be disappointing. The classifier just guessed the predominant class, Hamilton, every time. Looking at the train data shape above, there are 7876 unique words in the vocabulary. This may be too much, and many of those words may not be helpful. Redo the vectorization with max_features option set to use only the 1000 most frequent words. In addition to the words, add bigrams as a feature. Try Naïve Bayes again on the new train/test vectors and compare your results.**

---

In [234]:
vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS, max_features=1000, ngram_range=(1,2))
X_vtrain = vectorizer.fit_transform(X_train)
X_vtest = vectorizer.transform(X_test)
naive_bayes.fit(X_vtrain, y_train)
# make predictions on the test data
pred = naive_bayes.predict(X_vtest)
print(pred)
print('\nAccuracy score with Max Features 1000 and bigrams as a feature: ', accuracy_score(y_test, pred))

['HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON OR MADISON'
 'HAMILTON' 'HAMILTON' 'HAMILTON OR MADISON' 'JAY' 'MADISON' 'HAMILTON'
 'MADISON']

Accuracy score with Max Features 1000 and bigrams as a feature:  0.9411764705882353


---
**6. Try logistic regression. Adjust at least one parameter in the LogisticRegression() model to see if you can improve results over having no parameters. What are your results?**

---

In [235]:
pipe1 = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('logreg', LogisticRegression(class_weight='balanced'))
])

pipe1.fit(X_train, y_train)
pred=pipe1.predict(X_test)
print(pred)
print('\nAccuracy score for Logistic Regression: ', accuracy_score(y_test, pred))

['HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'HAMILTON' 'JAY' 'HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON'
 'HAMILTON' 'HAMILTON OR MADISON' 'JAY' 'MADISON' 'HAMILTON' 'HAMILTON']

Accuracy score for Logistic Regression:  0.9411764705882353


In [236]:
pipe1.steps

[('tfidf', TfidfVectorizer()),
 ('logreg', LogisticRegression(class_weight='balanced'))]

In [237]:
pipe1.named_steps['logreg'].get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [238]:
pipe1.named_steps['tfidf'].get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

Adjusting the parameter C. Increase it to reduce regularization

In [239]:
pipe1.set_params(logreg__C=2.0).fit(X_train, y_train)
pred = pipe1.predict(X_test)
print(pred)
print("\nAccuracy with C changed to 2.0: ", accuracy_score(y_test, pred))

['HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'HAMILTON' 'JAY' 'HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON'
 'HAMILTON' 'HAMILTON OR MADISON' 'JAY' 'MADISON' 'HAMILTON' 'MADISON']

Accuracy with C changed to 2.0:  1.0


---
**7. Try a neural network. Try different topologies until you get good results. What is your final accuracy?**

---

In [240]:
pipe1 = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('neuralnet', MLPClassifier(solver='lbfgs', alpha=1e-5,
                   hidden_layer_sizes=(15, 7), random_state=1234)),
         ])
pipe1.fit(X_train, y_train)
pred = pipe1.predict(X_test)
print(pred)
print("\nAccuracy with Neural Network: ", accuracy_score(y_test, pred))

['HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'MADISON' 'HAMILTON AND MADISON' 'MADISON' 'HAMILTON'
 'HAMILTON']

Accuracy with Neural Network:  0.6470588235294118


Lets add more hidden layers

In [241]:
pipe1 = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('neuralnet', MLPClassifier(solver='lbfgs', alpha=1e-5,
                   hidden_layer_sizes=(60, 30), random_state=1234)),
         ])
pipe1.fit(X_train, y_train)
pred = pipe1.predict(X_test)
print(pred)
print("Accuracy with more hidden layers: ", accuracy_score(y_test, pred))

['HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'HAMILTON' 'MADISON' 'HAMILTON' 'HAMILTON OR MADISON'
 'HAMILTON' 'HAMILTON' 'MADISON' 'JAY' 'MADISON' 'HAMILTON'
 'HAMILTON OR MADISON']
Accuracy with more hidden layers:  0.8235294117647058


Now let's increase the alpha regularization factor as well

In [242]:
pipe1 = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('neuralnet', MLPClassifier(solver='lbfgs', alpha=1e-2,
                   hidden_layer_sizes=(60, 30), random_state=1234)),
         ])
pipe1.fit(X_train, y_train)
pred = pipe1.predict(X_test)
print(pred)
print("accuracy: ", accuracy_score(y_test, pred))

['HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON' 'HAMILTON' 'HAMILTON'
 'HAMILTON' 'HAMILTON' 'JAY' 'HAMILTON' 'HAMILTON OR MADISON' 'HAMILTON'
 'HAMILTON' 'HAMILTON OR MADISON' 'JAY' 'MADISON' 'HAMILTON' 'MADISON']
accuracy:  1.0


With the above parameters we are getting a perfect accuracy of 100%